In [1]:
import time
from datetime import datetime

end_time = datetime.now().replace(hour=11, minute=2, second=0)
if datetime.now() > end_time:
    end_time = end_time.replace(day=end_time.day + 1)

while datetime.now() < end_time:
    time.sleep(60)

In [1]:
from trading_helper import TradingHelper
import time
import math
app=TradingHelper()
mail="cirigliano.santiago@gmail.com"
portfolio=app.portfolio(mail=mail)
tickers_in_portfolio=[]
for i in portfolio["tickers"]:
    ticker=i["ticker"]
    if ticker not in ['AR$', 'US$ MEP','AL30','GD30','TVPP']:
        tickers_in_portfolio.append(ticker)

tickers_in_portfolio=tuple(tickers_in_portfolio)
print(tickers_in_portfolio)
while True:
    for ticker in tickers_in_portfolio: 
        try:stocks_available=app.stocks_available(mail=mail,ticker=ticker,term="CI",currency="pesos")
        except:stocks_available=app.stocks_available(mail=mail,ticker=ticker,term="CI",currency="pesos")
        if stocks_available["CI"]>0:
            amount_in_portfolio=stocks_available["CI"]
            try:
                response = app.get_snapshot(mail=mail, ticker=ticker)
            except Exception as e:
                # Reintentar mientras el error sea un 500
                while "Error code: 500" in str(e):
                    try:
                        response = app.get_snapshot(mail=mail, ticker=ticker)
                        break  # Salir del bucle si la solicitud es exitosa
                    except Exception as e:
                        if "Error code: 500" not in str(e):
                            print(f"Error al obtener snapshot: {e}")  # Registro del error diferente
                            break  # Salir del bucle si es otro error
                        continue  # Continúa reintentando si es un error 500
            ci = [item for item in response if item['currency'] == 'ARS' and item['term'] == 'CI'][0]
            hs =  [item for item in response if item['currency'] == 'ARS' and item['term'] == '24hs'][0]
            #print(f"CI {ci}")
            #print(f"hs {hs}")
            precio_venta=ci['bids'][0]['price']
            vol_venta=ci['bids'][0]['size']
            precio_compra=hs['asks'][0]['price']
            vol_compra=hs['asks'][0]['size']
            minimum_vol = min(vol_compra,vol_venta,amount_in_portfolio)
            if minimum_vol == 0:break
            #Este bloque gestiona la cantidad de papeles a comprar y vender según dónde este la limitante del volumen
            #Maximiza la cantidad de recompra a 24 horas para hacer interes compuesto
            if minimum_vol == vol_venta:
                vol_compra = math.floor(vol_venta * (precio_venta/precio_compra))
            elif minimum_vol == amount_in_portfolio:
                vol_venta= amount_in_portfolio
                vol_compra = math.floor(vol_venta * (precio_venta/precio_compra))
            elif minimum_vol== vol_compra:
                vol_venta = math.floor(vol_compra /(precio_venta/precio_compra))
            #termina bloque de volumenes
            if precio_venta == 0 or precio_compra == 0: break
            ratio=precio_venta/precio_compra
            #print(f"RATIO {ratio} con {ticker}. Vendo {vol_venta} nominales y compro {vol_compra} nominales. venta CI a ${precio_venta} y compra 24hs a ${precio_compra}")
            rate=app.calculate_daily_rate(max_ratio=1.005,min_ratio=1.001,hora_negociacion=16)
            if ratio > rate and ratio < 1.2:
                print(f"RATIO {ratio} con {ticker}. Vendo {vol_venta} nominales y compro {vol_compra} nominales. venta CI a ${precio_venta} y compra 24hs a ${precio_compra}")
                sell_order=app.place_sell_order(mail=mail,
                                                ticker=ticker,
                                                quantity=vol_venta,
                                                price=precio_venta,
                                                term="CI",
                                                currency="pesos")
                print(f"Orden de venta {ticker} {sell_order}")
                order_status=app.check_order_status(mail=mail,
                                                    order_number=sell_order["Orden"])
                print(order_status)
                start_time = time.time()
                while order_status == "MARKET":
                    elapsed_time = time.time() - start_time
                    if elapsed_time > 4:
                        cancel_order=app.cancel_order(mail=mail,order_number=sell_order["Orden"])
                        print(f"Orden cancelada por Timeout: {cancel_order}")
                        break
                    order_status=app.check_order_status(mail=mail,
                                                    order_number=sell_order["Orden"])
                    print(f"Order Status de venta {order_status}")
                    
                if order_status == "EXECUTED":
                    buy_order=app.place_buy_order(mail=mail,
                                                ticker=ticker,
                                                quantity=vol_compra,
                                                price=precio_compra,
                                                term="24",
                                                currency="pesos")
                    print(f"Orden de compra {ticker} {buy_order}")
                    order_status=app.check_order_status(mail=mail,
                                                        order_number=buy_order["Orden"])
                    while order_status == "MARKET":
                        try:
                            time.sleep(1)
                            order_status=app.check_order_status(mail=mail, order_number=buy_order["Orden"])
                            if order_status =="FILLED": break
                            cancel_order=app.cancel_order(mail=mail,order_number=buy_order["Orden"])
                            precio_compra = precio_compra - 0.1
                            buy_order=app.place_buy_order(mail=mail,
                                ticker=ticker,
                                quantity=vol_compra,
                                price=precio_compra,
                                term="24",
                                currency="pesos")
                        except Exception as e: print(e)
                        
                    print(f"Order status de Compra {order_status}")
                    if buy_order["Success"] == True: print("Negocio ATR")

('BPOD7', 'CTIO', 'T2X5', 'PAMP', 'IRSA', 'TZX25', 'T5X4', 'TZX26', 'T4X4', 'S29N4', 'TX25', 'S13D4', 'CRES', 'TC25P', 'S17E5', 'TX26', 'S31E5', 'S28F5', 'GD41', 'TZV25')
